# Pytorch

Pytorch je Python bibliteka koja omogucava jednostavan rad sa neuronskim mrežama. Upustva za instalaciju možete pronaći na sljedećem linku:
https://pytorch.org/get-started/locally/
Ukoliko koristite Google Collab Pytorch će biti unaprijed instaliran. Jedna od glavnih prednosti rada sa Pytorch bibliotekom i tenzorima je mogućnost upotrebe grafiče kartice za ubrzavanje izračunavanja i automatsko računanje izvoda. Ako koristite Google Collab servis da biste omogućili upotrebu grafiče kartice potrebno je otići na opciju Edit te izabrati Notebook Settings. Unutar otvorenog modala potrebno je izabrati GPU u Hardware accelerator padajućem meniju. 

Kako bismo definisali neuronsku mrežu koristimo nn modul Pytorch biblioteke. Svaka mreža bi trebala na naslijedi klasu nn.Module koja implementira veliki broj korisnih funkcionalnosti. Unutar konstruktora mreže definišemo potrebne slojeve i aktivacione funkcije. Pored konstruktora potrebno je definisati i forward funkciju koja opisuje protok operacija od ulaza do izlaza. Jako je bitno obratiti pažnju na broj ulaznih i izlaznih neurona u svakom Linear sloju. U ulaznom sloju broj neurona mora biti jednak dimenzionalnosti ulaznog vektora dok broj neurona u krajnjem izlaznom sloju zavisi od samog problema, i tu razlikujemo tri slučaja:
1. regresija - izlazni sloj ima samo jedan neuron
2. binarna klasifikacija - izlazni sloj ima jedan neuron i sigmoid aktivacionu funkciju
3. klasifikacija u više klasa - broj izlaznih neurona je jednak broju klasa i koristimo softmax aktivaciju

In [1]:
import torch as T
import torch.nn as nn

In [2]:
class CaliforniaRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(8, 128)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(p=0.2)
        self.layer2 = nn.Linear(128, 128)
        self.act2 = nn.ReLU()
        self.drop2 = nn.Dropout(p=0.2)
        self.layer3 = nn.Linear(128, 64)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(64, 1)
 
    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.drop1(x)
        x = self.act2(self.layer2(x))
        x = self.drop2(x)
        x = self.act3(self.layer3(x))
        x = self.output(x)
        return x

In [3]:
from sklearn.datasets import fetch_california_housing

california = fetch_california_housing()
X = california.data
y = california.target

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [5]:
type(X_train)

numpy.ndarray

Podaci koji su učitani kroz sklearn.datasets modul koriste Numpy nizove. Pytorch radi sa tenzorima tako da je prvo potrebno konvertovati ulazne podatke u odgovarajući format. Bitno je napomenuti da je potrebno uraditi eksplicitno kastovanje podataka u float tip, jer slojevi neuronske mreže koriste vektore težina koji su float tipa. Pošto u datom primeru rješavamo problem regresije na izlazu imamo 1 neuron i podaci varijable $y$ koji su učitani kroz sklearn su predstavljeni jednodimenzionim nizom. Funkcija reshape(-1, 1) konvertuje niz brojeva u matricu tipa $Nx1$. Pytorch zahtijeva dati format kako bi se moglo lakše proširiti na rad sa prozivoljnim brojem neurona na izlazu.

In [6]:
from torch.utils.data import TensorDataset, DataLoader

X_train_t = T.tensor(X_train).float()
X_test_t = T.tensor(X_test).float()
y_train_t = T.tensor(y_train).float().reshape(-1, 1)
y_test_t = T.tensor(y_test).float().reshape(-1, 1)

In [7]:
print(X_train_t.shape)
print(y_train_t.shape)

torch.Size([16512, 8])
torch.Size([16512, 1])


In [8]:
# definisemo trening i testni dataset na osnovu tenzora
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

DataLoader klasa omogućava jednostavno učitavanje podataka u batchu na osnovu definisanog dataseta.

In [9]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

Prethodno smo rekli da Pytorch omogućava izvršavanje koda na grafičkoj kartici što značajno ubrzava treniranje mreže. Kako bismo omogućili datu funkcionalnost potrebno je definisati uređaj na kom će se izvršavati sve operacije. Nakon definisanja uređaja potrebno je i mrežu i sve podatke prebaciti na odgovarajući uređaj. Ukoliko se podaci ili mreža nalaze na različitim uređajima (jedan na CPU drugi na GPU) Pytorch ce javiti grešku.

In [10]:
device = T.device("cuda:0") if T.cuda.is_available() else T.device("cpu") # koristimo gpu samo ako je dostupan, inace koristimo cpu
print(device)

cpu


In [11]:
net = CaliforniaRegressor().to(device) # isntanciramo klasu mreže i prebacujemo je na odgovarajući uređaj

In [12]:
# metoda za testiranje mreže
# loss_fn predstavlja funkciju koja računa grešku
def test_model(net, loss_fn, test_loader):
    total_loss = 0
    net.eval() # prebacujemo mrežu u mod za testiranje/evaluaciju

    with T.no_grad(): # ne treba nam računanje gradijenta jer radimo u modu testiranja
        for x, y in test_loader: # iteriramo kroz dataloader, dobijamo podatke u batchu od 32
            x = x.to(device) # potrebno i x i y prebaciti na odgovarajući uređaj
            y = y.to(device)
            preds = net(x) # generišemo predikcije
            loss = loss_fn(preds, y) # računamo grešku
            total_loss += loss.item() # jako bitno pozvati .item() funkciju ako nam treba samo broj, jer loss čuva referencu na generisani graf izračunavanja koji nam nije potreban u ovom slučaju
        
    print(total_loss / len(test_loader))

In [13]:
# optimizer klasa čuva parametre mreže i omogućava njihovu promjenu na osnovu izračunatih gradijenata
# weight_decay opcija implementira regularizaciju
# Adam adaptivno mijenja parametar učenja tokom treninga što mu često daje bolje performanse od standardnog gradijetnog spusta
optimizer = T.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
#  Definisemo adekvatnu funkciju greske za problem
loss_fn = nn.MSELoss()

In [14]:
test_model(net, loss_fn, test_loader)

49.91852531137393


In [15]:
EPOCHS = 50

In [16]:
net.train() # stavljamo mrežu u mod rada za trening
for i in range(EPOCHS):
    if i % 5 == 0:
        print(f"Running epoch: {i}")
    
    # slicni koraci kao kod testiranja
    # ovaj put ne koristimo T.no_grad jer nam trebaju gradijenti
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        preds = net(x)
        loss = loss_fn(preds, y)
        # nuliramo prethodno izračunate gradijente
        optimizer.zero_grad()
        # prograpigramo grešku unazad
        loss.backward()
        # pravimo korak - mijenjamo parametre mreže na osnovu graijenta i faktora obučavanja
        optimizer.step()

Running epoch: 0
Running epoch: 5
Running epoch: 10
Running epoch: 15
Running epoch: 20
Running epoch: 25
Running epoch: 30
Running epoch: 35
Running epoch: 40
Running epoch: 45


In [17]:
test_model(net, loss_fn, test_loader)

0.48771673498689666
